In [1]:
#| default_exp notebook_context

## Jupyter Kernels detailed technical implementation

https://chatgpt.com/share/692bea08-4510-8004-b9ab-c02feeb97c08

## Build this extension

### Edit wordslab_notebooks_lib/__init__.py

Add this function definition:

```python
def _jupyter_labextension_paths():
    return [{
        "src": "labextension",
        "dest": "wordslab-notebooks-extension"
    }]
```

### Edit ???

Add this classifier for Pypi

"Framework :: Jupyter :: JupyterLab :: Extensions :: Prebuilt"

### Build everything

Open a Terminal

```bash
source activate-workspace-project wordslab-notebooks-lib

# Install dependencies
jlpm install

# Build TypeScript extension
jlpm run build

# Export notebooks to Python modules
nbdev_export

# Prepare for release
nbdev_prepare
```

### Test locally

```bash
source ../../jupyterlab/.venv/bin/activate

# Install in development mode
uv pip install -e .

# Register the extension with JupyterLab during development
jupyter labextension develop . --overwrite

# Verify extension is found
jupyter labextension list

# Start JupyterLab
jupyter lab
```

### When ready to publish:

```bash
source .venv/bin/activate

# Make sure extension is built
jlpm run build

# Publish to PyPI
nbdev_publish
```

In [1]:
#| export
import asyncio
from ipykernel.comm import Comm
from IPython import get_ipython

class NotebookContextManager:
    def __init__(self):
        self.comm_target = 'notebook_context_comm'
        self._comm = None
        self._registered = False
    
    def _ensure_registered(self):
        if not self._registered:
            self._comm = Comm(target_name=self.comm_target)
            self._registered = True
    
    async def get_notebook_context(self):
        self._ensure_registered()
        
        future = asyncio.Future()
        
        def handle_response(msg):
            if not future.done():
                future.set_result(msg['content']['data']['cells'])
        
        self._comm.on_msg(handle_response)
        self._comm.send({'action': 'request_cells'})
        
        try:
            result = await asyncio.wait_for(future, timeout=5.0)
            return result
        except asyncio.TimeoutError:
            raise TimeoutError("Failed to receive notebook context from frontend")
    
    def get_notebook_context_sync(self):
        try:
            import nest_asyncio
            nest_asyncio.apply()
        except ImportError:
            raise ImportError("nest_asyncio is required. Install with: pip install nest_asyncio")
        
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(self.get_notebook_context())
    
    def format_context_for_llm(self, cells, include_outputs=True):
        context_parts = []
        
        for i, cell in enumerate(cells):
            if cell['type'] == 'markdown':
                context_parts.append(f"## Markdown Cell {i+1}")
                context_parts.append(cell['source'])
            elif cell['type'] == 'code':
                context_parts.append(f"## Code Cell {i+1}")
                context_parts.append(f"```python\n{cell['source']}\n```")
                
                if include_outputs and cell['outputs']:
                    context_parts.append("### Output:")
                    for output in cell['outputs']:
                        if output.get('text'):
                            context_parts.append(output['text'])
                        elif output.get('data'):
                            data = output['data']
                            if 'text/plain' in data:
                                context_parts.append(data['text/plain'])
            
            context_parts.append("")
        
        return "\n".join(context_parts)

_manager = NotebookContextManager()

async def get_notebook_context():
    return await _manager.get_notebook_context()

def get_notebook_context_sync():
    return _manager.get_notebook_context_sync()

def format_for_llm(include_outputs=True):
    cells = _manager.get_notebook_context_sync()
    return _manager.format_context_for_llm(cells, include_outputs)


In [2]:
get_notebook_context_sync()

/tmp/ipykernel_9690/2177990006.py:14: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self._comm = Comm(target_name=self.comm_target)
Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "/home/python/cpython-3.12.12-linux-x86_64-gnu/lib/python3.12/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7ac34b311b40> is already entered
Task was destroyed but it is pending!
task: <Task pending name='Task-4' coro=<_async_in_context.<locals>.run_in_context() done, defined at /home/workspace/wordslab-notebooks-lib/.venv/lib/python3.12/site-packages/ipykernel/utils.py:57> wait_for=<Task pending name='Task-5' coro=<Kernel.shell_main() running at /home/workspace/wordslab-notebooks-lib/.venv/lib/python3.12/s

TimeoutError: Failed to receive notebook context from frontend

In [3]:
from ipykernel.kernelbase import Kernel

In [6]:
session = Kernel.instance().session

In [11]:
session.

180